In [3]:
"""
Steps to replicate derived property in holding domain on aggregation api returns different value. 

1. Define portfolio property. (which the derived property depends on)
2. Create portfolio with property.
3. upsert instruments.
4. Upsert transaction.
5. Upsert quote.
6. Upsert recipe.
7. Create holding derv property.
8. Do valuation to check values from these properties.
9. Check against holding API.

"""


"""
------------------------
Creating an API Factory:
------------------------

"""

# Import LUSID SDK
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
import logging
logger = logging.getLogger("")
logger.setLevel(logging.DEBUG)

# Import supporting LUSID utility packages
from lusidjam import RefreshingToken
#from fbnsdkutilities import ApiClientFactory

# Import other non-LUSID packages
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import pprint


# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")

logger.debug(f'Secrets path is {secrets_path}')

# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="DerivedPropertiesNotOnHoldingProviderTest"
)

logger.debug('LUSID Environment Initialised')

scope = "DERV_PROP_TEST"
transaction_scope = "DERV_PROP_TEST"
domain = "Portfolio"

# Time and date parameters in normalised format.
date_format = '%Y-%m-%d'


"""
---------------
Create Property
---------------

"""

# Create underlying value property in the portfolio domain.

properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_creation = properties_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain="Portfolio",
            scope=scope,
            code='Schema',
            display_name='Schema',
            data_type_id=models.ResourceId(code="string", scope="system"),
        )
    )

    logger.debug(property_creation)

except lusid.ApiException as e:
    logger.debug(json.loads(e.body)["title"])

# Create derived property that uses this value property.
derived_property_code = 'HoldingFromPortfolioDerivedProperty'
derived_property_formula = f"if ( Properties[Portfolio/DERV_PROP_TEST/Schema] eq 'JC_UK_Demo2_Schema' )\nthen 'IsJC_UK_Demo2_Schema'\nelse 'NotJC_UK_Demo2_Schema'"

try:
    property_creation = properties_api.create_derived_property_definition(
        create_derived_property_definition_request=models.CreateDerivedPropertyDefinitionRequest(
            domain="Holding",
            scope=scope,
            code=derived_property_code,
            display_name=derived_property_code,
            data_type_id=models.ResourceId(code="string", scope="system"),
            derivation_formula = derived_property_formula
        )
    )

    logger.debug(property_creation)

except lusid.ApiException as e:
    logger.debug(json.loads(e.body)["title"])

In [6]:
"""
GetHoldings call without underlying portfolio property.

Derived property wrongly evaluates to false on condition.
"""

txn_portfolio_api = api_factory.build(lusid.TransactionPortfoliosApi)

def get_holdings_with_properties(portfolio_scope, portfolio_code, property_key_list):
    try:
        get_holdings = txn_portfolio_api.get_holdings(
            scope = 'Demo2',
            code = 'JC_UK',
            property_keys = [
                'Holding/DERV_PROP_TEST/HoldingFromPortfolioDerivedProperty'
            ]
        )

        pprint.pprint(get_holdings)

    except lusid.ApiException as e:
        logger.debug(json.loads(e.body)["title"])

holdings_properties = [
    'Holding/DERV_PROP_TEST/HoldingFromPortfolioDerivedProperty'
]

get_holdings_with_properties("Demo2", "JC_UK", holdings_properties)

{'href': 'https://fbn-joe.lusid.com/api/api/transactionportfolios/Demo2/JC_UK/holdings?effectiveAt=2024-09-10T09%3A20%3A59.8697456%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/portfolios/Demo2/JC_UK',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/PortfolioHolding',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/properties?propertyKeys=Holding%2FDERV_PROP_TEST%2FHoldingFromPortfolioDerivedProperty%2CHolding%2Fdefault%2FSourcePortfolioId%2CHolding%2Fdefault%2FSourcePortfolioScope',
            'method': 'GET',
            'relation': 'PropertySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
          

In [5]:
"""
With referenced property
"""

holdings_properties.append('Portfolio/DERV_PROP_TEST/Schema')

get_holdings_with_properties("Demo2", "JC_UK", holdings_properties)

{'href': 'https://fbn-joe.lusid.com/api/api/transactionportfolios/Demo2/JC_UK/holdings?effectiveAt=2024-09-10T09%3A20%3A53.4308641%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/portfolios/Demo2/JC_UK',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/PortfolioHolding',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/properties?propertyKeys=Portfolio%2FDERV_PROP_TEST%2FSchema%2CHolding%2FDERV_PROP_TEST%2FHoldingFromPortfolioDerivedProperty%2CHolding%2Fdefault%2FSourcePortfolioId%2CHolding%2Fdefault%2FSourcePortfolioScope',
            'method': 'GET',
            'relation': 'PropertySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'log